In [13]:
import pymongo
from pprint import pprint

In [2]:
# Connecting to the Server
conn = pymongo.MongoClient('localhost', 27017)

# Creating or connectin to the Database
db = conn['big_data_class']

# Creating or connecting to the Collection
coll = db.amazon_sample

In [ ]:
# Filter by Customer
query = {'Customer': 'Ram Prasad' }
pprint(list(coll.find(query))) 

In [35]:
# Filter by '_id'
coll.find_one({'_id':'C3'})

{'_id': 'C3',
 'Customer': 'Raelynn Dodson',
 'Gender': 'F',
 'Age': 44,
 'phone_number': {'$numberLong': '443482883256'},
 'addresses': [{'_id': 'AD3',
   'house': 3,
   'street': 'Dickinson St',
   'city': 'Manchester',
   'post_code': 'M1 4LF',
   'location': {'type': 'Point', 'coordinates': [53.47850891, -2.242348226]}}],
 'current_orders': [{'_id': '20230104035938C3',
   'date': {'$date': {'$numberLong': '1672804778000'}},
   'order_status': 2,
   'order_details': [{'product_id': 'PH1', 'quantity': 1, 'cost': 169}],
   'total_cost': 169,
   'partner_id': '',
   'shipping_id': 'AD3',
   'supplier_id': 'W1'},
  {'_id': '20230104045148C3',
   'date': {'$date': {'$numberLong': '1672807908000'}},
   'order_status': 1,
   'order_details': [{'product_id': 'FP2', 'quantity': 1, 'cost': 2.7},
    {'product_id': 'FP1', 'quantity': 5, 'cost': 2}],
   'total_cost': 210,
   'partner_id': '',
   'shipping_id': 'AD3',
   'supplier_id': 'ST4'},
  {'_id': '20230104054212C3',
   'date': {'$date': {

In [19]:
# Filter by Age > 30
query = { 'Age' : {'$gt':30}}
pprint(list(coll.find(query)))

[{'Age': 51,
  'Customer': 'Gunner Ferrell',
  'Gender': 'M',
  '_id': 'C1',
  'addresses': [{'_id': 'AD1',
                 'city': 'Manchester',
                 'house': 27,
                 'location': {'coordinates': [53.4702888668, -2.26459207339],
                              'type': 'Point'},
                 'post_code': 'M15 4RU',
                 'street': 'Ellesmere St'}],
  'current_orders': [{'_id': '20230104002920C1',
                      'date': {'$date': {'$numberLong': '1672792160000'}},
                      'order_details': [{'cost': 2,
                                         'product_id': 'FP1',
                                         'quantity': 4},
                                        {'cost': 2.7,
                                         'product_id': 'FP2',
                                         'quantity': 1}],
                      'order_status': 3,
                      'partner_id': '',
                      'shipping_id': 'AD1',
                 

In [71]:
# Filter count the number of Customers with Age > 30
query = {'Age': {'$gt':30} }
pprint(coll.count_documents(query)) # Count

14


In [38]:
# Get the average age of the customers
pipeline = [
        {'$group': {
            '_id':None, # to group everything into one
            'average_age': {'$avg' : '$Age'} # creating the average variable
        }}
    ]

pprint( list(coll.aggregate(pipeline)) )

[{'_id': None, 'average_age': 44.1}]


In [62]:
#####################

### DOES NOT WORK ###

#####################


# $first returns the first document instance and NOT the correct customer with highest purchase


projection = {
    '_id': 0,
    'Customer': 1,
    'highest purchase': 1
}

pipeline = [
        { '$unwind': '$current_orders' },
        {'$group': {
            '_id':None, # to group everything into one
            'Customer' : { '$first': '$Customer'}, # $first returns the first document instance and NOT the correct customer with highest 
            'highest purchase': {'$max' : '$current_orders.total_cost'} # creating the average variable
        }},
        {
            '$project': projection
        }
    ]

pprint( list(coll.aggregate(pipeline)) )

[{'Customer': 'Gunner Ferrell', 'highest purchase': 2437.4}]


In [50]:
#####################

##### DOES WORK #####

#####################

projection = {
    '_id': 0,
    'Customer': 1,
    'highest purchase': {'$max' : '$current_orders.total_cost'} 
}

sort = {
    'highest purchase': -1
}

limit = {
    '$limit': 1
}

pipeline = [
        {
            '$project': projection
        },
        {
            '$sort': sort
        },
        {**limit}

    ]

pprint( list(coll.aggregate(pipeline)) )

[{'Customer': 'Ram Prasad', 'highest purchase': 2437.4}]


In [69]:
#####################

##### DOES WORK #####

#####################


# This uses $reduce

projection = {
    '_id': 0,
    'Customer': 1,
    'highest purchase': {'$reduce' : {
        'input':'$current_orders',
        'initialValue': 0,
        'in': {
            '$cond':{
                'if': { '$gt': ['$$this.total_cost', '$$value']},
                'then': '$$this.total_cost',
                'else': '$$value'
            }
        }
    }} 
}

pipeline = [
        {
            '$project': projection
        },
        {
            '$sort': {'highest purchase': -1}
        },
        {
            '$limit': 1
        }
    ]

lst = list(coll.aggregate(pipeline))

if lst:
    pprint( lst)





[{'Customer': 'Ram Prasad', 'highest purchase': 2437.4}]


In [80]:
#  Get the total amount of product quantities sold by product using $unwind and $sort
pipeline = [
        {'$unwind': '$current_orders'},
        {'$unwind': '$current_orders.order_details'},
        {'$group': {
            "_id" : "$current_orders.order_details.product_id",
            'total_quantity': {'$sum': "$current_orders.order_details.quantity"}
        }},
        {
            '$sort' : {'_id' : 1}
        }
    ]

pprint( list(coll.aggregate(pipeline)) )

[{'_id': 'B3', 'total_quantity': 1},
 {'_id': 'B4', 'total_quantity': 2},
 {'_id': 'B6', 'total_quantity': 1},
 {'_id': 'B7', 'total_quantity': 8},
 {'_id': 'B9', 'total_quantity': 1},
 {'_id': 'CD1', 'total_quantity': 1},
 {'_id': 'CD10', 'total_quantity': 9},
 {'_id': 'CD2', 'total_quantity': 1},
 {'_id': 'CD5', 'total_quantity': 2},
 {'_id': 'CD6', 'total_quantity': 1},
 {'_id': 'CD8', 'total_quantity': 1},
 {'_id': 'CD9', 'total_quantity': 2},
 {'_id': 'FP1', 'total_quantity': 16},
 {'_id': 'FP10', 'total_quantity': 10},
 {'_id': 'FP12', 'total_quantity': 5},
 {'_id': 'FP13', 'total_quantity': 2},
 {'_id': 'FP2', 'total_quantity': 5},
 {'_id': 'FP3', 'total_quantity': 10},
 {'_id': 'FP4', 'total_quantity': 4},
 {'_id': 'FP5', 'total_quantity': 6},
 {'_id': 'FP7', 'total_quantity': 4},
 {'_id': 'FP9', 'total_quantity': 9},
 {'_id': 'HA1', 'total_quantity': 1},
 {'_id': 'HA3', 'total_quantity': 2},
 {'_id': 'HA5', 'total_quantity': 1},
 {'_id': 'HA6', 'total_quantity': 1},
 {'_id': '

In [86]:
# Total purchase by customer
pipeline = [
        {'$project': {
            "_id" : "$_id",
            'Customer' : "$Customer",
            'Total purchase': {'$sum': "$current_orders.total_cost"}
        }},
        { '$sort': { 'Customer': 1 }}
    ]

pprint( list(coll.aggregate(pipeline)) )

[{'Customer': 'Alyvia Mccormick', 'Total purchase': 157.37, '_id': 'C16'},
 {'Customer': 'Aubree Holland', 'Total purchase': 205.4, '_id': 'C18'},
 {'Customer': 'Billy Ross', 'Total purchase': 11.46, '_id': 'C10'},
 {'Customer': 'Brice Copeland', 'Total purchase': 15.46, '_id': 'C14'},
 {'Customer': 'Calvin Carlson', 'Total purchase': 43.92, '_id': 'C15'},
 {'Customer': 'Charity Dunn', 'Total purchase': 18.09, '_id': 'C7'},
 {'Customer': 'Gunner Ferrell', 'Total purchase': 26.7, '_id': 'C1'},
 {'Customer': 'Jairo Fitzgerald', 'Total purchase': 26, '_id': 'C11'},
 {'Customer': 'Jamarcus Montgomery', 'Total purchase': 14, '_id': 'C6'},
 {'Customer': 'Kelsey Brandt', 'Total purchase': 48.65, '_id': 'C8'},
 {'Customer': 'Kylee Wang', 'Total purchase': 28.47, '_id': 'C9'},
 {'Customer': 'Lillie Costa', 'Total purchase': 23.6, '_id': 'C2'},
 {'Customer': 'Lucian Cohen', 'Total purchase': 26, '_id': 'C12'},
 {'Customer': 'Mark Mulldon', 'Total purchase': 863, '_id': 'C20'},
 {'Customer': 'Pat

In [93]:
# Average purchase
pipeline = [
        {'$group': {
            "_id" : None,
            'Total purchase': {'$avg': {'$sum': "$current_orders.total_cost"}}
        }}
    ]

pprint( list(coll.aggregate(pipeline)) )

[{'Total purchase': 276.094, '_id': None}]
